In [1]:
import torch
import random
import torchvision
import numpy as np
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from sklearn.model_selection import train_test_split

from utils.util import get_image_list, dataset_list, PathDataset, PathDataset_v2, make_task

from avalanche.training.supervised import LwF
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [4]:
data_path = "../Data/CASIA-WebFace/" 

dataset = get_image_list(data_path)
x_train, x_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.2, shuffle=True, random_state=seed, stratify=dataset[1])

train_set = dataset_list(x_train, y_train)
test_set = dataset_list(x_test, y_test)

print("train : ", len(y_train))
print("test : ", len(y_test))

train :  395531
test :  98883


In [5]:
transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 10575
incremental = 5
# train_task = make_task(incremental, y_train)
# test_task = make_task(incremental, y_test)

# train_set = PathDataset(train_set, transforms_group, "train", train_task)
# test_set = PathDataset(test_set, transforms_group, "eval", test_task)
train_set = PathDataset_v2(train_set, transforms_group, "train")
test_set = PathDataset_v2(test_set, transforms_group, "eval")

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=True,
                        seed=seed,
                        shuffle=False
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-1)
criterion = torch.nn.CrossEntropyLoss()

In [7]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(project_name="Face Recognition", run_name="GEM-CASIA-M10-E10")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hopo55. Use `wandb login --relogin` to force relogin


/home/cal-06/anaconda3/envs/hspark/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


## GEM

In [8]:
train_batch = 10
eval_batch = 1
epoch = 10
memory_size = 10

strategies = GEM(model, optimizer, criterion, patterns_per_exp=memory_size, memory_strength=0.5, train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [9]:
print("Starting experiment...")
results = []
for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)
    strategies.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategies.eval(scenario.test_stream[:]))

Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|██████████| 11693/11693 [09:47<00:00, 19.89it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0285
-- >> End of training phase << --
End training on experience  0
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 29221/29221 [06:11<00:00, 78.59it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 525.4676
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 5.8297
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp000 = 39.0000
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp000 = 4671.2852
	Time_Exp/eval_phase/test_stream/Task000/Exp000 = 371.8229
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0687
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 20168/20168 [